In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import math


In [ ]:
class Self_Attention(nn.Module):
    def __init__(self,n_heads:int,d_embed,in_porj=True,out_porj=True):
        super().__init__()
        self.n_heads=n_heads
        self.d_embed=d_embed
        self.in_porj=nn.Linear(d_embed,d_embed*3,bias=in_porj)
        self.out_porj=nn.Linear(d_embed,d_embed,bias=out_porj)
        self.d_heads=d_embed//n_heads
    def forward(self,x:torch.Tensor,mask=None):
        # x batch,channels,w,h
        # w*h represent seq_len=each_pixcel and channels diffine its embeddig cause each channels capture different property=wieghts 

        inshape=x.shape
        batch_size,seq_len,embedding=inshape
        interim_shape=batch_size,seq_len,self.n_heads,self.d_heads
        q,k,v=self.in_porj(x).chunk(3,-1)
        
        q=q.view(interim_shape).transpose(1,2)
        k=k.view(interim_shape).transpose(1,2)
        v=v.view(interim_shape).transpose(1,2)
        
        weight=q@k.transpose(-1,-2)
        if mask:
            mask=torch.ones_like(weight,dtype=torch.bool).triu(1)
            weight.masked_fill(mask,-torch.inf)
        weight/=math.sqrt(self.d_heads)
        weight=F.softmax(weight,dim=-1)
        out_put=weight@v
        out_put=self.out_porj(out_put)
        return out_put

        


In [16]:
n=torch.rand(1,4,256,4)
k=torch.rand(1,4,256,4).transpose(-1,-2)
z=n@k
print(z.shape)
v=torch.rand(1,4,256,4)
o=z@v
o.shape

torch.Size([1, 4, 256, 256])


torch.Size([1, 4, 256, 4])